In [1]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines
from utilities import *
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner
from transformers.trainer_callback import TrainerCallback
import itertools
from IPython.display import clear_output

In [2]:
model_name = "EleutherAI/pythia-410m"
#model_name = "EleutherAI/pythia-160m"
#model_name = "EleutherAI/pythia-70m"
# Get the current directory
current_directory = os.getcwd()
# Join the folder path
folder_path = os.path.join(current_directory, "content")
dataset_name = "ai-medical-chatbot_processed.jsonl"
dataset_path = os.path.join(folder_path, dataset_name)
#dataset_path = f"/content/{dataset_name}"
use_hf = False
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": use_hf,
        "path": dataset_path
    },
    "verbose": True
}
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
train_dataset, test_dataset = tokenize_and_split_data(training_config, tokenizer)
base_model = AutoModelForCausalLM.from_pretrained(model_name)
device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

c:\Blog\How-to-Finetuning-Large-Language-Models\.venv\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\066226758\.cache\huggingface\hub\models--EleutherAI--pythia-410m. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

2024-04-10 15:24:15,379 - DEBUG - utilities - Config: datasets.path: c:\Blog\How-to-Finetuning-Large-Language-Models\content\ai-medical-chatbot_processed.jsonl
datasets.use_hf: false
model.max_length: 2048
model.pretrained_name: EleutherAI/pythia-410m
verbose: true



...

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
2024-04-10 15:24:15,952 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-067cad356d1dba73/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/tmpuj4mco1o
2024-04-10 15:24:17,273 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-067cad356d1dba73/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/tmpv5b1up9z
2024-04-10 15:24:17,289 - DEBUG - fsspec.local - open file: C:/Users/066226758/.cache/huggingface/datasets/json/default-067cad356d1dba73/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/tmpp_8jyaij


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/911M [00:00<?, ?B/s]

2024-04-10 15:26:58,316 - DEBUG - utilities - Select CPU device


In [3]:
from collections import Counter
import math

def cosine_similarity(str1, str2):
    """
    Computes the cosine similarity between two strings using the Bag-of-Words model.

    Args:
        str1: The first string.
        str2: The second string.

    Returns:
        A float representing the cosine similarity between the two strings.
    """
    # Tokenize the strings
    tokens1 = str1.split()
    tokens2 = str2.split()

    # Create bag of words for each string
    bow1 = Counter(tokens1)
    bow2 = Counter(tokens2)

    # Get the set of all unique words
    all_words = set(bow1.keys()).union(set(bow2.keys()))

    # Compute dot product
    dot_product = sum(bow1[word] * bow2[word] for word in all_words)

    # Compute magnitudes
    magnitude1 = math.sqrt(sum(bow1[word] ** 2 for word in all_words))
    magnitude2 = math.sqrt(sum(bow2[word] ** 2 for word in all_words))

    # Compute cosine similarity
    if magnitude1 == 0 or magnitude2 == 0:
        return 0
    else:
        return dot_product / (magnitude1 * magnitude2)


In [4]:
def inference_new(text, model, tokenizer, max_input_tokens=1000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
      text,
      return_tensors="pt",
      truncation=True,
      max_length=max_input_tokens
  )

  # Generate
  device = model.device
  attention_mask = torch.ones_like(input_ids)  # Create mask with all 1s

  # Fix: Mask all padding tokens, including the first element
  attention_mask[input_ids == tokenizer.pad_token_id] = 0

  generated_tokens_with_prompt = model.generate(
      input_ids.to(device),
      max_length=max_output_tokens,
      attention_mask=attention_mask,
      pad_token_id=tokenizer.eos_token_id  # Set pad token
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]
  return generated_text_answer



In [5]:
from transformers import AutoModelForCausalLM
import shutil
def train_model(hyperparameters, delete=False, testing=False):
  max_steps = hyperparameters["max_steps"]


  # Convert hyperparameter values to integers and add them to the string
  hyperparameter_str = '_'.join(str(int(value)) if isinstance(value, (int, float)) else value for value in hyperparameters.values())
  # Create the trained_model_name variable
  trained_model_name = f"ai_medical_{hyperparameter_str}"

  #trained_model_name = f"ai_medical_{max_steps}_steps"
  output_dir = trained_model_name
  training_args = TrainingArguments(
    # Learning rate
    learning_rate=hyperparameters["learning_rate"],

    # Number of training epochs
    num_train_epochs=hyperparameters["num_train_epochs"],

    # Max steps to train for (each step is a batch of data)
    # Overrides num_train_epochs, if not -1
    max_steps=max_steps,

    # Batch size for training
    per_device_train_batch_size=hyperparameters["per_device_train_batch_size"],

    # Directory to save model checkpoints
    output_dir=output_dir,

    # Other arguments
    overwrite_output_dir=False, # Overwrite the content of the output directory
    disable_tqdm=False, # Disable progress bars
    eval_steps=120, # Number of update steps between two evaluations
    save_steps=120, # After # steps model is saved
    warmup_steps=1, # Number of warmup steps for learning rate scheduler
    per_device_eval_batch_size=1, # Batch size for evaluation
    evaluation_strategy="steps",
    logging_strategy="steps",
    logging_steps=1,
    optim=hyperparameters["optim"],
    gradient_accumulation_steps = hyperparameters['gradient_accumulation_steps'],
    gradient_checkpointing=False,
    # Parameters for early stopping
    load_best_model_at_end=True,
    save_total_limit=1,
    metric_for_best_model="eval_loss",
    greater_is_better=False
  )
  base_model.to(device)
  model_flops = (
    base_model.floating_point_ops(
      {
        "input_ids": torch.zeros(
            (1, training_config["model"]["max_length"])
        )
      }
    )
    * training_args.gradient_accumulation_steps
  )

  #print(base_model)
  print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
  print("Flops", model_flops / 1e9, "GFLOPs")

  trainer = Trainer(
    model=base_model,
    model_flops=model_flops,
    total_steps=max_steps,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)
  training_output = trainer.train()
  # Evaluate the model
  eval_results = trainer.evaluate()

  # Adding Evaluation 
  save_dir = f'{output_dir}'
  trainer.save_model(save_dir)
  print("Saved model to:", save_dir)
  finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(save_dir, local_files_only=True)
  finetuned_slightly_model.to(device)
  test_question = test_dataset[0]['question']
  print("Question input (test):", test_question)
  predicted_answer=inference_new(test_question, finetuned_slightly_model, tokenizer)
  print("Finetuned slightly model's answer: ")
  print(predicted_answer) 
  test_answer = test_dataset[0]['answer']
  print("Target answer output (test):", test_answer)
  metric_cosine_similarity=cosine_similarity(test_answer, predicted_answer)
  print("Cosine Similarity:", metric_cosine_similarity)
  # Deleting the folder to save space
  clear_output()
  if delete:
    shutil.rmtree(save_dir)
    print("Deleted model folder:", save_dir)
  if testing:
    return eval_results, training_output, metric_cosine_similarity,test_question,test_answer,predicted_answer

  else:
    return eval_results, training_output, metric_cosine_similarity


In [6]:
hyperparameters={'learning_rate': 1e-06,
'num_train_epochs': 1,
'per_device_train_batch_size': 1,
'optim': 'adafactor',
'num_iterations': 1,
'max_steps':3,
'gradient_accumulation_steps':2}

In [7]:
eval_results, training_output, metric_cosine_similarity,test_question,test_answer,predicted_answer =train_model(hyperparameters, delete=True, testing=True)

Deleted model folder: ai_medical_0_1_1_adafactor_1_3_2


In [8]:
test_text = test_dataset[0]['question']
print("Question input (test):", test_text)
print(f"Correct answer from ai-medical-chatbot: {test_dataset[0]['answer']}")
print("Model's answer: ")
print(inference_new(test_text, base_model, tokenizer))

Question input (test): Will Kalarchikai cure multiple ovarian cysts in PCOD?
Correct answer from ai-medical-chatbot: Hello. I just read your query. See Kalarachi Kai choornam is helpful in amenorrhea. As far as small cysts are concerned they are unmatured eggs which failed to induce menstrual cycle previously, as a result, they got collected in the ovary and they will remain in the ovary. Now, you have got your periods you can start trying for conception. But I advise you to do it under the supervision of a nearby gynecologist because egg size is important while conception and that you can know by ovulation study. Ovulation study is performed under the supervision of a gynecologist. For gall stones, surgical intervention is required generally. Medicine is not of much help.
Model's answer: 


In [ ]:
from tqdm import tqdm
import pandas as pd

In [ ]:
def find_best_hyperparameters():
    best_hyperparameters = None
    best_loss = float('inf')
    # Lists to store data
    hyperparameters_list = []
    eval_results_list = []
    training_output_list = []
    cosine_similarity_list = []

    test_question_list = []
    test_answer_list = []
    predicted_answer_list = []



    
    # Define hyperparameter search space
    hyperparameter_space = {
        "learning_rate": [6.0e-5, 3.0e-4],
        "num_train_epochs": [1,5],
        "per_device_train_batch_size": [1],
        "optim": ["adafactor"],
        "max_steps": [1,10],
        "gradient_accumulation_steps": [2],
    }

    # Generate all combinations of hyperparameters
    all_hyperparameters = list(itertools.product(*hyperparameter_space.values()))

    # Assuming all_hyperparameters is a list of hyperparameter combinations
    for hyperparameter_values in tqdm(all_hyperparameters):
        hyperparameters = dict(zip(hyperparameter_space.keys(), hyperparameter_values))
        
        # Evaluate the model
        # Print the current hyperparameters
        print("Using hyperparameters:")
        for key, value in hyperparameters.items():
            print(f"{key}: {value}")
        eval_results, training_output, metric_cosine_similarity, test_question, test_answer, predicted_answer = train_model(hyperparameters,delete=True,testing=True)
        
        # Append data to lists
        hyperparameters_list.append(hyperparameters)
        eval_results_list.append(eval_results)
        training_output_list.append(training_output)
        cosine_similarity_list.append(metric_cosine_similarity)

        test_question_list.append(test_question)
        test_answer_list.append(test_answer)
        predicted_answer_list.append(predicted_answer)

        # Check if this set of hyperparameters gives better results
        if eval_results["eval_loss"] < best_loss:
                best_loss = eval_results["eval_loss"]
                best_hyperparameters = hyperparameters
        clear_output()
    # Create DataFrame
    data = {
        'Question':test_question_list,
        'Answer':test_answer_list,
        'Prediction':predicted_answer_list,
        'Hyperparameters': hyperparameters_list,
        'Evaluation Results': eval_results_list,
        'Training Output': training_output_list,
        'Cosine Similarity': cosine_similarity_list
    }
    df = pd.DataFrame(data)
    
    return best_hyperparameters, best_loss, df

In [ ]:
# Call the function to find the best hyperparameters
best_hyperparameters, best_loss ,df = find_best_hyperparameters()

100%|██████████| 8/8 [17:06<00:00, 128.33s/it]


In [ ]:
print("Best hyperparameters:", best_hyperparameters)
print("Best loss:", best_loss)

Best hyperparameters: {'learning_rate': 6e-05, 'num_train_epochs': 1, 'per_device_train_batch_size': 1, 'optim': 'adafactor', 'max_steps': 20, 'gradient_accumulation_steps': 2}
Best loss: 2.4853429794311523


In [ ]:
# Sort the DataFrame by 'eval_loss' inside the 'Evaluation Results' column
df_sorted = df.sort_values(by='Evaluation Results', 
                           key=lambda x: x.apply(lambda d: d['eval_loss']))



In [ ]:
df_sorted.head()

,Question,Answer,Prediction,Hyperparameters,Evaluation Results,Training Output,Cosine Similarity
0,Will Kalarchikai cure multiple ovarian cysts i...,Hello. I just read your query. See Kalarachi K...,Hi. I have gone through your query and have go...,"{'learning_rate': 6e-05, 'num_train_epochs': 1...","{'eval_loss': 2.4853429794311523, 'eval_runtim...","(20, 3.3880001068115235, {'train_runtime': 47....",0.144474
1,Will Kalarchikai cure multiple ovarian cysts i...,Hello. I just read your query. See Kalarachi K...,Hello. I have gone through your query and test...,"{'learning_rate': 6e-05, 'num_train_epochs': 5...","{'eval_loss': 2.5364460945129395, 'eval_runtim...","(20, 2.0024101391434668, {'train_runtime': 49....",0.211604
2,Will Kalarchikai cure multiple ovarian cysts i...,Hello. I just read your query. See Kalarachi K...,Hello. I have gone through your query and test...,"{'learning_rate': 6e-05, 'num_train_epochs': 1...","{'eval_loss': 2.7494125366210938, 'eval_runtim...","(20, 1.4947877436876298, {'train_runtime': 47....",0.345688
3,Will Kalarchikai cure multiple ovarian cysts i...,Hello. I just read your query. See Kalarachi K...,Hello. I just read your query. I would like yo...,"{'learning_rate': 6e-05, 'num_train_epochs': 2...","{'eval_loss': 3.0984041690826416, 'eval_runtim...","(20, 1.1360332652926446, {'train_runtime': 45....",0.265248
4,Will Kalarchikai cure multiple ovarian cysts i...,Hello. I just read your query. See Kalarachi K...,Hi. For further information consult a gynecolo...,"{'learning_rate': 0.0003, 'num_train_epochs': ...","{'eval_loss': 4.455532073974609, 'eval_runtime...","(20, 5.409056072868407, {'train_runtime': 43.8...",0.233142


In [ ]:
df_sorted.head(1)["Prediction"].values

array(['Hi. I have gone through your query and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gone through your queries and have gon

In [ ]:
# Sort the DataFrame by 'Cosine Similarity' from largest to smallest
df_cos = df.sort_values(by='Cosine Similarity', ascending=False)

In [ ]:
df_cos[["Answer","Prediction","Cosine Similarity","Evaluation Results"]]

,Answer,Prediction,Cosine Similarity,Evaluation Results
2,Hello. I just read your query. See Kalarachi K...,Hello. I have gone through your query and test...,0.345688,"{'eval_loss': 2.7494125366210938, 'eval_runtim..."
3,Hello. I just read your query. See Kalarachi K...,Hello. I just read your query. I would like yo...,0.265248,"{'eval_loss': 3.0984041690826416, 'eval_runtim..."
4,Hello. I just read your query. See Kalarachi K...,Hi. For further information consult a gynecolo...,0.233142,"{'eval_loss': 4.455532073974609, 'eval_runtime..."
6,Hello. I just read your query. See Kalarachi K...,Hello. I have gone through your information an...,0.229964,"{'eval_loss': 5.980817794799805, 'eval_runtime..."
1,Hello. I just read your query. See Kalarachi K...,Hello. I have gone through your query and test...,0.211604,"{'eval_loss': 2.5364460945129395, 'eval_runtim..."
0,Hello. I just read your query. See Kalarachi K...,Hi. I have gone through your query and have go...,0.144474,"{'eval_loss': 2.4853429794311523, 'eval_runtim..."
5,Hello. I just read your query. See Kalarachi K...,Hi. I am here to help you. For further informa...,0.107587,"{'eval_loss': 5.183531284332275, 'eval_runtime..."
7,Hello. I just read your query. See Kalarachi K...,Hello. She would not get ovarian cysts. She wo...,0.075293,"{'eval_loss': 6.59088134765625, 'eval_runtime'..."
